In [1]:
"""  
word embeddings are dense vecotrs of real numbers , 

how should you represent a word in a computers? 
ascii character : only tell you what the word is. doesn't what means

V : our vocabulary if one-hot encoding : |v| elements |0,0,....,1,....0|
where 1 is in a location unique to w.

getting dense word embeddings

similarity(physics, mathematics) = Q_p * Q_m / || Q_p || * ||Q_m|| 
= cos(phi)

phi : angle between the two cevtors that extremly similar word will have 1, dissimilar should have -1

 summary, word embeddings are a representation of the *semantics* of a word, efficiently encoding semantic information that might be relevant to the task at hand. You can embed other things too: part of speech tags, parse trees, anything! The idea of feature embeddings is central to the field.

"""
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)


In [2]:
word_to_ix = {"hello":0,"world":1}
embeds = nn.Embedding(2,5) # 2words in in vocab 5 dimensinal embeddings

lookup_tensor = torch.tensor([word_to_ix["hello"]],dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


In [9]:
sentences = 'hello word your name'.split()
sentences

['hello', 'word', 'your', 'name']

In [21]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [[test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)]+[test_sentence[i + j + 1] for j in range(CONTEXT_SIZE)]],
        test_sentence[i],i
    )
    for i in range(CONTEXT_SIZE, len(test_sentence)-2)
]

ngrams

[([['forty', 'When', 'shall', 'besiege']], 'winters', 2),
 ([['winters', 'forty', 'besiege', 'thy']], 'shall', 3),
 ([['shall', 'winters', 'thy', 'brow,']], 'besiege', 4),
 ([['besiege', 'shall', 'brow,', 'And']], 'thy', 5),
 ([['thy', 'besiege', 'And', 'dig']], 'brow,', 6),
 ([['brow,', 'thy', 'dig', 'deep']], 'And', 7),
 ([['And', 'brow,', 'deep', 'trenches']], 'dig', 8),
 ([['dig', 'And', 'trenches', 'in']], 'deep', 9),
 ([['deep', 'dig', 'in', 'thy']], 'trenches', 10),
 ([['trenches', 'deep', 'thy', "beauty's"]], 'in', 11),
 ([['in', 'trenches', "beauty's", 'field,']], 'thy', 12),
 ([['thy', 'in', 'field,', 'Thy']], "beauty's", 13),
 ([["beauty's", 'thy', 'Thy', "youth's"]], 'field,', 14),
 ([['field,', "beauty's", "youth's", 'proud']], 'Thy', 15),
 ([['Thy', 'field,', 'proud', 'livery']], "youth's", 16),
 ([["youth's", 'Thy', 'livery', 'so']], 'proud', 17),
 ([['proud', "youth's", 'so', 'gazed']], 'livery', 18),
 ([['livery', 'proud', 'gazed', 'on']], 'so', 19),
 ([['so', 'livery'

In [23]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}



[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]
{'winters': 0, 'new': 1, 'own': 2, 'lies,': 3, 'being': 4, 'This': 5, 'days;': 6, 'old,': 7, 'How': 8, 'trenches': 9, 'see': 10, "excuse,'": 11, 'all-eating': 12, 'treasure': 13, 'lusty': 14, 'child': 15, 'be': 16, 'Where': 17, 'brow,': 18, 'beauty': 19, 'thou': 20, 'shall': 21, "deserv'd": 22, 'it': 23, 'mine': 24, 'Shall': 25, 'And': 26, 'of': 27, "youth's": 28, 'sum': 29, 'made': 30, 'when': 31, 'succession': 32, 'thine!': 33, 'by': 34, 'Proving': 35, 'use,': 36, 'much': 37, 'in': 38, 'sunken': 39, 'old': 40, 'and': 41, 'asked,': 42, 'my': 43, 'worth': 44, 'thriftless': 45, "beauty's": 46, 'Will': 47, 'weed': 48, 'within': 49, 'Were': 50, 'say,': 51, 'besiege': 52, 'so': 53, 'fair': 54, 'were': 55, "totter'd": 56, 'make': 57, 'on': 58, 'to': 59, 'a': 60, 'art': 61, 'deep': 62, 'count,': 63, 'blood': 64, 'answer': 65, "feel'st": 66, 'thy': 67, 'small': 68, 'eyes,': 69, 'the': 70, 'now

In [5]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]
[522.0125954151154, 519.4978120326996, 517.0001952648163, 514.5184724330902, 512.0522618293762, 509.60040259361267, 507.1605079174042, 504.73329162597656, 502.3170166015625, 499.9118301868439]
tensor([-1.1051, -0.6999,  0.2366,  1.9130,  1.8377,  1.3237, -0.0713,  0.3472,
        -0.6522,  1.5573], grad_fn=<SelectBackward0>)


In [6]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# Create your model and train. Here are some functions to help you make
# the data ready for use by your module.


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

[(['are', 'We', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]


tensor([44, 20,  2, 43])